# EOCanvas Batch Processing with C2RCC on SNAP

## Setup

In [1]:
from eocanvas import API, Credentials
from eocanvas.api import Input, Config, ConfigOption
from eocanvas.processes import SnapProcess
from eocanvas.snap.graph import Graph

In [2]:
c = Credentials.load()

In [3]:
from hda import Client

In [4]:
c = Client()

In [5]:
c.metadata("EO:ESA:DAT:SENTINEL-2")

{'type': 'object',
 'title': 'Querable',
 'properties': {'dataset_id': {'title': 'dataset_id',
   'type': 'string',
   'oneOf': [{'const': 'EO:ESA:DAT:SENTINEL-2',
     'title': 'EO:ESA:DAT:SENTINEL-2',
     'group': None}]},
  'bbox': {'title': 'Bbox',
   'type': 'array',
   'minItems': 4,
   'maxItems': 4,
   'items': [{'type': 'number', 'maximum': 180, 'minimum': -180},
    {'type': 'number', 'maximum': 90, 'minimum': -90},
    {'type': 'number', 'maximum': 180, 'minimum': -180},
    {'type': 'number', 'maximum': 90, 'minimum': -90}]},
  'productIdentifier': {'title': 'Product Identifier',
   'type': 'string',
   'pattern': '^[a-zA-Z0-9]+$'},
  'productType': {'title': 'Product Type',
   'type': 'string',
   'oneOf': [{'const': 'S2MSI1C', 'title': 'S2MSI1C', 'group': None},
    {'const': 'S2MSI2A', 'title': 'S2MSI2A', 'group': None},
    {'const': 'AUX_GNSSRD', 'title': 'AUX_GNSSRD', 'group': None},
    {'const': 'AUX_PROQUA', 'title': 'AUX_PROQUA', 'group': None},
    {'const': 'AU

In [12]:
q = {
    "dataset_id": "EO:ESA:DAT:SENTINEL-2",
    "startdate": "",
    "enddate": "",
    "processingLevel": "S2MSI1C",
    "tileId": ""
}

In [6]:
import json

In [7]:
with open('AOIs.txt', 'r') as fp:
    poly_dict = json.load(fp)

In [8]:
poly_dict

{'2': {'geoRegion': 'POLYGON ((-5.22444257 49.9901843, -4.88225479 49.9901843, -4.88225479 50.29003327, -5.22444257 50.29003327, -5.22444257 49.9901843))',
  'tile_Id': '30UUA'},
 '3': {'geoRegion': 'POLYGON ((-4.81917128 50.19269224, -4.59558622 50.19269224, -4.59558622 50.40467681, -4.81917128 50.40467681, -4.81917128 50.19269224))',
  'tile_Id': '30UUA'},
 '4': {'geoRegion': 'POLYGON ((-4.32949749 50.26538565, -4.02885352 50.26538565, -4.02885352 50.5075424, -4.32949749 50.5075424, -4.32949749 50.26538565))',
  'tile_Id': '30UVA'},
 '6': {'geoRegion': 'POLYGON ((-4.02836874 50.14143345, -3.59050258 50.14143345, -3.59050258 50.34954708, -4.02836874 50.34954708, -4.02836874 50.14143345))',
  'tile_Id': '30UVA'},
 '9': {'geoRegion': 'POLYGON ((-3.70217003 50.32171356, -3.4070158 50.32171356, -3.4070158 50.54736093, -3.70217003 50.54736093, -3.70217003 50.32171356))',
  'tile_Id': '30UVA'},
 '11': {'geoRegion': 'POLYGON ((-4.42178593 50.96017888, -3.80799343 50.96017888, -3.80799343 51.

In [10]:
tile_polys = {}
for v in poly_dict.values():
    tile_id = v['tile_Id']
    geo_region = v['geoRegion']

    if tile_id not in tile_polys:
        tile_polys[tile_id] = []
    tile_polys[tile_id].append(geo_region)

In [11]:
tile_polys

{'30UUA': ['POLYGON ((-5.22444257 49.9901843, -4.88225479 49.9901843, -4.88225479 50.29003327, -5.22444257 50.29003327, -5.22444257 49.9901843))',
  'POLYGON ((-4.81917128 50.19269224, -4.59558622 50.19269224, -4.59558622 50.40467681, -4.81917128 50.40467681, -4.81917128 50.19269224))',
  'POLYGON ((-5.7897289 49.9734232, -5.3357743 49.9734232, -5.3357743 50.27582106, -5.7897289 50.27582106, -5.7897289 49.9734232))'],
 '30UVA': ['POLYGON ((-4.32949749 50.26538565, -4.02885352 50.26538565, -4.02885352 50.5075424, -4.32949749 50.5075424, -4.32949749 50.26538565))',
  'POLYGON ((-4.02836874 50.14143345, -3.59050258 50.14143345, -3.59050258 50.34954708, -4.02836874 50.34954708, -4.02836874 50.14143345))',
  'POLYGON ((-3.70217003 50.32171356, -3.4070158 50.32171356, -3.4070158 50.54736093, -3.70217003 50.54736093, -3.70217003 50.32171356))'],
 '30UVB': ['POLYGON ((-4.42178593 50.96017888, -3.80799343 50.96017888, -3.80799343 51.4427289, -4.42178593 51.4427289, -4.42178593 50.96017888))',
 

In [12]:
import csv

In [15]:
tile_dates = {}

In [16]:
with open("query_dates.csv", "r") as fp:
    reader = csv.reader(fp, delimiter=",")
    next(reader)
    for tileId, startdate, enddate in reader:
        tile_dates[tileId] = {'startdate': startdate, 'enddate': enddate}

In [17]:
tile_dates

{'30UUA': {'startdate': '2020-06-23T00:00:00.000Z',
  'enddate': '2020-06-24T00:00:00.000Z'},
 '30UVA': {'startdate': '2020-06-23T00:00:00.000Z',
  'enddate': '2020-06-24T00:00:00.000Z'},
 '30UVB': {'startdate': '2020-06-23T00:00:00.000Z',
  'enddate': '2020-06-24T00:00:00.000Z'},
 '30UWB': {'startdate': '2020-06-25T00:00:00.000Z',
  'enddate': '2020-06-26T00:00:00.000Z'},
 '30UXB': {'startdate': '2020-07-30T00:00:00.000Z',
  'enddate': '2020-07-31T00:00:00.000Z'},
 '30UWC': {'startdate': '2020-06-25T00:00:00.000Z',
  'enddate': '2020-06-26T00:00:00.000Z'},
 '29UPR': {'startdate': '2020-06-01T00:00:00.000Z',
  'enddate': '2020-06-02T00:00:00.000Z'},
 '30UUC': {'startdate': '2020-06-01T00:00:00.000Z',
  'enddate': '2020-06-02T00:00:00.000Z'},
 '30UVC': {'startdate': '2020-06-23T00:00:00.000Z',
  'enddate': '2020-06-24T00:00:00.000Z'},
 '31UCS': {'startdate': '2023-07-07T00:00:00.000Z',
  'enddate': '2023-07-08T00:00:00.000Z'},
 '31UCT': {'startdate': '2023-07-07T00:00:00.000Z',
  'endda

## Process workflow

In [ ]:
with open("s2_c2rcc.xml", "r") as fp:
    graph_file = fp.read()

In [ ]:
config = Config(key="img1", options=ConfigOption(uncompress=False, sub_path=""))

In [ ]:
for k, v in tile_polys.items():

In [ ]:
q['tileId'] = k
r = c.search(q)

In [ ]:
process = SnapProcess(snap_graph=graph, eo_config=config, eo_input=inputs)

In [ ]:
modified_xml = graph_file.replace("$polygon", geo_region)
graph = Graph.from_text(modified_xml)